In [2]:
## change to our functinon
from stage import getStage, fullPrint
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    tickerIndex = random.randint(0,len(yf.tickers_nasdaq())-1)
    for symbol in yf.tickers_nasdaq():
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        print(index, allocation, total, len(element.holding))
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    print(transactionCopy.iloc[-1]['total'])
    transactionCopy.to_csv("transaction/checkBug.csv")
    transactionCopy.to_pickle("transaction/checkBug.pkl")
    return transactionCopy.iloc[-1]['total']
evalOneMax([[0.19078223936143746, 0.986675953424998, 0.07057270627935228, 0.9528588558753942, 1.0282875027195482, 1.251806657541451, 0.823960510068463, 1.1414927855254375]])

[0.19078223936143746, 0.986675953424998, 0.07057270627935228, 0.9528588558753942, 1.0282875027195482, 1.251806657541451, 0.823960510068463, 1.1414927855254375]
1999-07-26 33.333333333333336 100 3
1999-08-02 32.90546140318482 98.71638420955446 3
1999-08-09 21.180671510555452 105.90335755277727 5
1999-08-16 51.0440134068232 102.0880268136464 2
1999-08-23 98.40719196677857 98.40719196677857 1
1999-08-30 25.41995043376756 101.67980173507024 4
1999-09-06 21.273413149535866 106.36706574767933 5
1999-09-13 14.941696702946833 104.59187692062784 7
1999-09-20 32.47638191166399 97.42914573499198 3
1999-09-27 45.75984569970544 91.51969139941087 2
1999-10-04 33.60966663254704 100.82899989764111 3
1999-10-11 19.162429174404345 95.81214587202172 5
1999-10-18 25.615201342400965 102.46080536960386 4
1999-10-25 33.247116621127034 99.7413498633811 3
1999-11-01 17.87175915657347 107.23055493944082 6
1999-11-08 11.58230711109765 115.8230711109765 10
1999-11-15 8.824175033057681 123.53845046280753 14
1999-1

16711.474903258782

In [1]:
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("result.csv")
from stageOld import getFullDfOld
getFullDfOld("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("resultOld.csv")

In [1]:
# from stage import getStage
# getStage("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544])
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544]).to_csv("result.csv")

2000-02-28 00:00:00 0.888392984867096
0.888392984867096
2000-03-20 00:00:00 1.1160709857940674
1.1160709857940674
